In [2]:
import pickle

hr_sent_dict = pickle.load(open("/home/ether/hdd/ether/graph_dumps/hr/hr_sent_dict_v3.pkl","rb"))
hr_com_map = pickle.load(open("com_map_hr.pkl","rb"))
hr_com_ranked = pickle.load(open("ranked_com_hr.pkl", "rb"))

In [27]:
rank_1_com_hr = list(map(lambda kv:kv[0], sorted(hr_com_ranked.items(), key=lambda kv:kv[1], reverse=False)[:4]))

In [28]:
rank_1_com_hr_ent = [ent for ent in hr_com_map if hr_com_map[ent] in rank_1_com_hr]

In [29]:
import sys
import numpy as np
sys.path.append("/home/arjun/BERT_Similarity_experiments/code/")
from gpt_feat_utils import GPT_Inference
gpt_model = GPT_Inference("/home/shubham/projects/domain_minds_v2_gpt/se/model/epoch3/", device="cpu")


se_fv_of_hr_ent = {}
for ent in rank_1_com_hr_ent:
        if ent in hr_sent_dict.keys():
            se_fv_of_hr_ent[ent] = np.mean([gpt_model.get_text_feats(sent) for sent in hr_sent_dict[ent]], axis=0)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [30]:
# S.E entity info

ent_fv_full = pickle.load(open("/home/arjun/NER_experiments/code/entity_graph_builder/graph_dumps/se_entity_feats_se_model_v2epc3.pkl", "rb"))

for ent, fv in se_fv_of_hr_ent.items():
    ent_fv_full[ent] = fv 

In [31]:
pickle.dump(ent_fv_full ,open("mind_data/se_mixed_hr_feats.pkl", "wb"))

In [32]:
se_fv_of_hr_ent.keys()

dict_keys(['Toyota', 'HR', 'W', 'Jessica', '__url__', 'Post', 'College', 'Blog', 'Michael', 'DNC', 'Truth', 'L', 'PC', 'Kris', 'SHRM', 'Ann Bares', 'Ann', 'Turkey', 'KD', 'Darth Vader', 'Ricky', 'Jon', 'Mike', 'CFO', 'BTW', 'J', 'Wall Street Journal', 'SuccessFactors', 'Dave Ulrich', '--matt', 'Polish', 'Joe', 'Change Agile', 'Hi Kris', 'Great Post', 'Tech', 'HR Department', 'KPI', 'HCM', 'Josh', 'Conference', 'SHRM Florida', 'Frank', 'Adidas', 'Spandex', 'Angela', 'Android', 'Scoreboard', 'Alot', 'Seth Godin', 'Seth', 'Mr', 'Service', 'Gladwell', 'HP', 'Human Capital', 'C', 'Blackberry', 'Mom', 'BlackBerry', 'Iphone', 'Meg', 'Steve Jobs', 'Ipod', 'HR Pro', 'ER', 'Dr', 'Noah', 'Wally', 'Karen', 'Matt', 'Jason', 'ROWE', 'Chris', 'Mac', 'LD', 'X', 'Free', 'Paul', 'Cliff', 'G', 'HR Pros', 'Interview', 'McCain', 'Exec', 'Industry', 'Re', 'Marc', 'Lars Dalgaard', 'Grad', 'Jon Ingham', 'Sue Meisinger', 'Sue', 'Execs', 'Thanks', 'Nice', 'Chrome', 'Viking', 'Roberts', 'Green', 'Jackson', 'Org'

# Clustering S.E org Cluster

In [5]:
import pickle

se_com_map = pickle.load(open("/home/ether/hdd/ether/mind_dumps/se/com_map_se.pkl", "rb"))
ent_fv_full = pickle.load(open("/home/ether/hdd/ether/mind_dumps/se/se_entity_feats.pkl", "rb"))

In [20]:
import statistics
import numpy as np
from copy import deepcopy
full_weights = []
def prune_edge(graph):
    c_weight = 0
    import statistics
    max_connection = {}
    max_score = {}
    outlier_score = {}
    most_similar= {}
    for node in graph.nodes():
        closest_connection_n = sorted(dict(graph[node]).items(), key=lambda kv:kv[1]["weight"], reverse=True)
        most_similar[node] = (closest_connection_n)
        weights_n = list(map(lambda kv: (kv[1]["weight"]).tolist(), closest_connection_n))
        q3 = np.percentile(weights_n, 75)
        #iqr = np.subtract(*np.percentile(weights_n, [75, 25]))
        outlier_score[node] = {}
        #outlier_score[node]["outlier"] = q3 + 1 * iqr
        #outlier_score[node]["iqr"] = iqr
        outlier_score[node]["q3"] = q3
        #outlier_score[node]["weights_n"] = closest_connection_n
        #outlier_score[node]["avg+pstd"] = statistics.mean(weights_n)+statistics.pstdev(weights_n)
        
    print (graph.number_of_edges())
    graph_data = deepcopy(graph.edges.data())
    print (len(graph_data))
    for nodea, nodeb, weight in graph_data:
        if weight["weight"] >= outlier_score[nodea]["q3"]:
            #print (outlier_score[nodea]["q3"], ":  ", weight["weight"])
            pass
        else:
            #print (outlier_score[nodea]["q3"], ":  ", weight["weight"])
            graph.remove_edge(nodea, nodeb)
        #print ("Pruning done for node:", nodea)
    print (graph.number_of_edges())
    return graph, most_similar
    

In [29]:
import networkx as nx
from scipy.spatial.distance import cosine

G = nx.Graph()
org_ent = [ent for ent in se_com_map if se_com_map[ent] ==0]

G.add_nodes_from(org_ent)

for index1, nodea in enumerate(G.nodes()):
    for index2, nodeb in enumerate(G.nodes()):
        G.add_edge(nodea, nodeb, weight=1-cosine(ent_fv_full[nodea], ent_fv_full[nodeb]))
        
G_pruned, cls = prune_edge(G)

330891
330891
85722


In [30]:
from community import best_partition

com = best_partition(G_pruned, resolution=0.9)

In [31]:
prev = 0 
com_map = {}
for ent, cls in sorted(com.items(),key=lambda kv:kv[1]):
    if prev!=cls:
        print ("----------------------Cluster-----------------")
        prev = cls
    
    print (ent)
    
    if cls in com_map.keys():
        com_map[cls].append(ent)
    else:
        com_map[cls] = [ent]

Awesome
Facebook Messenger
Center
Admin
Itunes
Google Ads
Alexa
Dream
Instagram
Yld
Emojis
Visio
Stackexchange
Halloween
Hipchat
Confluence
Hacktoberfest
Squarespace
Telegram
Tinder
Spotifys
Tumblr
Visa
Valve
Bing
Dropbox
Youtube
Asana
Ses
Company
Appsflyer
Marketplace
W3c
Joomla
Slack
Basecamp 3
Hackernews
Ether
Google Doc
Unsplash
Big
Qwiklabs
Wikis
Paypal
Wiki
Pyro
Cms
Shopify
Bluehost
Google Docs
Slackbot
Usp
Basecamp
Sharepoint
Excel
Rethinkdb
Soundcloud
E-mail
Bubble
Wix
Seo
Groupon
Linkedins
Uber
Crm
Twitters
Hacker News
Newsroom
Idea
Mailchimp
Boom
Intranet
Woocommerce
Kerberos
Google
Zendesk
Salesforce
Onenote
Jboss
Atlassian
Evernote
Buffer
Ecommerce
Word
E-commerce
Yelp
News
Source
Gmail
Noteworthy
Myspace
Dazn
Online
Commento
Getambassadorio
Google Adwords
Blog
Chime
Reddit
Knowledge Graph
Wordpress
Microsoft Excel
Pinterest
Medium
Basecamp 2
Fortnite
Google Search Engine
Spotify
Google Tag Manager
Webex
Dms
Onedrive
Fedex
Platform
Social
Slack Channel
Google Lighthouse
Goo

In [ ]:
from networkx.algorithms.vitality import closeness_vitality
vt = closeness_vitality(G_pruned)